In [1]:
import xarray as xr
import numpy as np

xr.set_options(display_style="text", display_expand_data=False)
rng = np.random.default_rng(seed=0)

In [3]:
lat = np.arange(-90, 90, 0.1)
lon = np.arange(0, 360, 0.1)

lat2d, lon2d = np.meshgrid(lat, lon)

data = rng.standard_normal(lat2d.shape)
arr = xr.DataArray(
    data,
    dims=("x", "y"),
    coords={"lat": (("x", "y"), lat2d), "lon": (("x", "y"), lon2d)},
)
arr

<xarray.DataArray (x: 3600, y: 1800)>
0.1257 -0.1321 0.6404 0.1049 -0.5357 ... 1.813 1.865 -0.1664 0.6568 -0.5354
Coordinates:
    lat      (x, y) float64 -90.0 -89.9 -89.8 -89.7 ... 89.6 89.7 89.8 89.9
    lon      (x, y) float64 0.0 0.0 0.0 0.0 0.0 ... 359.9 359.9 359.9 359.9
Dimensions without coordinates: x, y

In [3]:
arr.xindexes

Indexes:

In [4]:
from scipy.spatial import KDTree


class KDTreeIndex(xr.core.indexes.Index):
    def __init__(self, data, names, dim, **options):
        self.names = names
        self.dim = dim
        self.shape = data.shape
        self.index = KDTree(data.reshape(-1, self.shape[-1]), **options)

    @classmethod
    def from_variables(cls, variables, **options):
        data = np.concatenate(
            [var.data[..., None] for var in variables.values()], axis=-1
        )
        dims = {var.dims for var in variables.values()}
        if len(dims) != 1:
            raise ValueError("variables need to have the same dimensions")
        (dims,) = dims
        names = list(variables.keys())
        return cls(data, names, dims, **options)

    def sel(self, indexers):
        unknown_dimensions = set(indexers) - set(self.names)
        if unknown_dimensions:
            raise ValueError("unknown dimensions:", list(unknown_dimensions))

        points = np.concatenate(
            [indexers[name][..., None] for name in self.names],
            axis=-1,
        )

        distances, indices_ = self.index.query(points)
        indices = np.unravel_index(indices_, self.shape[:-1])

        isel_indexers = {
            dim: xr.DataArray(
                data, coords={"distance": ("points", distances)}, dims="points"
            )
            for dim, data in zip(self.dim, indices)
        }

        return xr.core.indexes.IndexSelResult(isel_indexers)

In [5]:
tree = KDTreeIndex.from_variables(arr.coords)
tree

In [6]:
new_arr = xr.DataArray(
    arr.variable,
    coords={
        "lat": xr.Variable(("x", "y"), lat2d),
        "lon": xr.Variable(("x", "y"), lon2d),
    },
    indexes={"lat": tree, "lon": tree},
    fastpath=True,
)
new_arr

<xarray.DataArray (x: 3600, y: 1800)>
0.5236 1.364 1.516 0.3978 -1.871 1.009 ... -0.2158 -1.338 -0.6278 2.928 -1.114
Coordinates:
  * lat      (x, y) float64 -90.0 -89.9 -89.8 -89.7 ... 89.6 89.7 89.8 89.9
  * lon      (x, y) float64 0.0 0.0 0.0 0.0 0.0 ... 359.9 359.9 359.9 359.9
Dimensions without coordinates: x, y

In [7]:
new_arr.xindexes

Indexes:
lat: <__main__.KDTreeIndex object at 0x7f90f82968b0>
lon: <__main__.KDTreeIndex object at 0x7f90f82968b0>

In [8]:
indexers = {
    "lat": np.array([0.742, 10.213, 17.648]),
    "lon": np.array([8.873, 3.12, 9.15]),
}

In [9]:
new_arr.sel(indexers)

<xarray.DataArray (points: 3)>
0.2345 -1.423 -0.519
Coordinates:
    lat       (points) float64 0.7 10.2 17.6
    lon       (points) float64 8.9 3.1 9.1
    distance  (points) float64 0.04993 0.02385 0.06931
Dimensions without coordinates: points